In [1]:
!pip install gymnasium
!pip install moviepy
!pip install pysdl2
!pip install pyvirtualdisplay

In [2]:
!pip install opencv-python

In [51]:
import gymnasium as gym
import random 
import torch
import numpy as np
from torch import nn
from torch.distributions import Categorical

from gymnasium import wrappers
import matplotlib.pyplot as plt
from IPython import display
from tqdm.notebook import tqdm

env = gym.make("ALE/Pong-v5", render_mode="rgb_array")
# record the game as as an mp4 file
#env = wrappers.RecordVideo(env, 'video/pong-base')

In [52]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [53]:
ACTIONS = [2, 3]

In [54]:
def render(env):
    img.set_data(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [59]:
import torch
import torch.nn as nn
import torch.optim as optim

class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(6000, 200),
            nn.ReLU(),
            nn.Linear(200, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, inputs):
        return self.main(inputs)

In [60]:
def preprocess_state(image):
    """ prepro 210x160x3 uint8 frame into 6000 (80x80) 1D float vector """
    image = image[35:185] # crop
    image = image[::2,::2,0] # downsample by factor of 2
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else (paddles, ball) just set to 1
    x = torch.from_numpy(image.astype(np.float32).ravel()).unsqueeze(0)
    return x

In [61]:
from tqdm.notebook import tqdm

model = PolicyNet()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
with tqdm(range(10000)) as pbar:
    average = -21
    
    for episode in pbar:
        state, _ = env.reset()
        prev_obs = None
        cur_obs = preprocess_state(state)
        rewards = []
        log_probs = []
        for t in range(10000):
            obs = cur_obs - prev_obs if prev_obs is not None else torch.zeros(6000).unsqueeze(0)
            probs = model(obs)
            action = np.random.choice([0, 1], p=probs.detach().numpy()[0])
            log_prob = torch.log(probs[0][action])
            # get next state and preprocess it
            state, reward, done, *_ = env.step(ACTIONS[action])
            prev_obs = cur_obs
            cur_obs = preprocess_state(state)
            # remember rewards and log probs
            rewards.append(reward)
            log_probs.append(log_prob)
            if done:
                break


    
        score = sum(rewards)
        average = 0.7 * average + (1 - 0.7)*score
        pbar.set_description(f"score: {score} average: {average:.2f}")
        loss = 0

        gamma=0.99  # Discounting factor
        running_add = 0
        rewards=np.array(rewards)
        discounted_r = np.zeros_like(rewards)
        for t in reversed(range(0, rewards.size)):
            if rewards[t] != 0: # reset the sum, since this was a game boundary (pong specific!)
                running_add = 0
            running_add = running_add * gamma + rewards[t]
            discounted_r[t] = running_add
        
        for log_prob, reward in zip(log_probs, discounted_r):
            loss = loss + -log_prob * reward
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

env.close()

  0%|          | 0/10000 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

